## 환경 설정

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirect
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292697 sha256=7a5a435c12194ffcb3c74ed32379c887e6126398600561e35a193de6cc0cc324
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-mlj795fe/kobert-tokenizer_40566d567fdd4284b838826fdd8bb761
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-mlj795fe/kobert-tokenizer_

In [2]:
!pip install mecab-python3
!pip install konlpy
!pip install tensorflow-text
!pip install soynlp
!pip install -v python-mecab-ko
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 3.4 MB/s eta 0:00:00
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 23.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import json
import os
import re
from konlpy.tag import Kkma
from soynlp.normalizer import *
from soynlp.tokenizer import LTokenizer
from konlpy.tag import Komoran
from transformers import BertTokenizer
import mecab
import sentencepiece as spm

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [6]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

## 데이터

In [7]:
train = pd.read_excel('/content/drive/MyDrive/WCRC/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.xlsx')
train.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [8]:
val = pd.read_excel('/content/drive/MyDrive/WCRC/018.감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation/감성대화말뭉치(최종데이터)_Validation.xlsx')
val.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.


## 전처리

In [9]:
train_df1 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1']]
train_df2 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장2', '시스템문장2']]
train_df3 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장3', '시스템문장3']]

# 열 이름 변경
train_df1 = train_df1.rename(columns={'시스템문장1': '시스템문장', '사람문장1': '사람문장'})
train_df2 = train_df2.rename(columns={'시스템문장2': '시스템문장', '사람문장2': '사람문장'})
train_df3 = train_df3.rename(columns={'시스템문장3': '시스템문장', '사람문장3': '사람문장'})

train_df = pd.concat([train_df1, train_df2, train_df3], ignore_index=True)

# NaN 값을 가진 행 삭제
train_df = train_df.dropna(subset=['사람문장', '시스템문장'])

# Unnamed: 0 기준으로 정렬
train_df = train_df.sort_values(by='Unnamed: 0')

# 인덱스 재설정
train_df = train_df.reset_index(drop=True)

train_df

,Unnamed: 0,상황키워드,감정_대분류,감정_소분류,사람문장,시스템문장
0,1,"진로,취업,직장",분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,1,"진로,취업,직장",분노,노여워하는,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.
2,2,"진로,취업,직장",분노,노여워하는,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.
3,2,"진로,취업,직장",분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
4,3,"진로,취업,직장",분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
...,...,...,...,...,...,...
145949,51629,대인관계,불안,걱정스러운,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?
145950,51629,대인관계,불안,걱정스러운,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
145951,51630,대인관계,상처,배신당한,남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.,대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.
145952,51630,대인관계,상처,배신당한,사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야.,어떻게 하면 지금의 상황에 변화를 만들어낼 수 있을까요?


In [10]:
val_df1 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1']]
val_df2 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장2', '시스템문장2']]
val_df3 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장3', '시스템문장3']]

val_df1 = val_df1.rename(columns={'시스템문장1': '시스템문장', '사람문장1': '사람문장'})
val_df2 = val_df2.rename(columns={'시스템문장2': '시스템문장', '사람문장2': '사람문장'})
val_df3 = val_df3.rename(columns={'시스템문장3': '시스템문장', '사람문장3': '사람문장'})

val_df = pd.concat([val_df1, val_df2, val_df3], ignore_index=True)
val_df = val_df.dropna(subset=['사람문장', '시스템문장'])
val_df = val_df.sort_values(by='Unnamed: 0')
val_df = val_df.reset_index(drop=True)

In [11]:
train_df['상황키워드'].value_counts()

대인관계            31376
진로,취업,직장        14606
재정              14584
건강,죽음           12340
가족관계            11768
연애,결혼,출산        11282
대인관계(부부, 자녀)     9641
학업 및 진로          9020
학교폭력/따돌림         8464
건강               8452
재정,은퇴,노후준비       7626
직장, 업무 스트레스      6795
Name: 상황키워드, dtype: int64

In [12]:
train_df.loc[(train_df['상황키워드'] == "대인관계"), '상황키워드'] = 0
train_df.loc[(train_df['상황키워드'] == "진로,취업,직장"), '상황키워드'] = 1
train_df.loc[(train_df['상황키워드'] == "재정"), '상황키워드'] = 2
train_df.loc[(train_df['상황키워드'] == "건강,죽음"), '상황키워드'] = 3
train_df.loc[(train_df['상황키워드'] == "가족관계"), '상황키워드'] = 4
train_df.loc[(train_df['상황키워드'] == "연애,결혼,출산"), '상황키워드'] = 5
train_df.loc[(train_df['상황키워드'] == "대인관계(부부, 자녀)"), '상황키워드'] = 6
train_df.loc[(train_df['상황키워드'] == "학업 및 진로"), '상황키워드'] = 7
train_df.loc[(train_df['상황키워드'] == "학교폭력/따돌림"), '상황키워드'] = 8
train_df.loc[(train_df['상황키워드'] == "건강"), '상황키워드'] = 9
train_df.loc[(train_df['상황키워드'] == "재정,은퇴,노후준비"), '상황키워드'] = 10
train_df.loc[(train_df['상황키워드'] == "직장, 업무 스트레스"), '상황키워드'] = 11

In [13]:
val_df.loc[(val_df['상황키워드'] == "대인관계"), '상황키워드'] = 0
val_df.loc[(val_df['상황키워드'] == "진로,취업,직장"), '상황키워드'] = 1
val_df.loc[(val_df['상황키워드'] == "재정"), '상황키워드'] = 2
val_df.loc[(val_df['상황키워드'] == "건강,죽음"), '상황키워드'] = 3
val_df.loc[(val_df['상황키워드'] == "가족관계"), '상황키워드'] = 4
val_df.loc[(val_df['상황키워드'] == "연애,결혼,출산"), '상황키워드'] = 5
val_df.loc[(val_df['상황키워드'] == "대인관계(부부, 자녀)"), '상황키워드'] = 6
val_df.loc[(val_df['상황키워드'] == "학업 및 진로"), '상황키워드'] = 7
val_df.loc[(val_df['상황키워드'] == "학교폭력/따돌림"), '상황키워드'] = 8
val_df.loc[(val_df['상황키워드'] == "건강"), '상황키워드'] = 9
val_df.loc[(val_df['상황키워드'] == "재정,은퇴,노후준비"), '상황키워드'] = 10
val_df.loc[(val_df['상황키워드'] == "직장, 업무 스트레스"), '상황키워드'] = 11

In [14]:
from konlpy.tag import Okt
import mecab

# Okt
okt = Okt()

# Mecab
mecab = mecab.MeCab()

# SentencePiece BPE Tokenization
sp = spm.SentencePieceProcessor()
sp.Load('/content/drive/MyDrive/WCRC/03.AI모델/04.참조모델/Pre모델링/A092_albert_mediazen/spm-mediazen.model')

True

In [15]:
def preprocess_data(text):
    # 정규화
    text = okt.normalize(text)

    # 한글, 물음표, 느낌표, 쉼표, 공백을 제외한 나머지 문자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ?!,\s]', '', text)

    # Mecab 형태소 분석
    # mecabbed_text = " ".join(mecab.morphs(text))

    # SentencePiece BPE Tokenization
    # sp_pieces = sp.EncodeAsPieces(mecabbed_text)

    return text

In [16]:
# '사람문장' 열에 대해 전처리 적용
train_df['사람문장_전처리'] = train_df['사람문장'].apply(preprocess_data)
val_df['사람문장_전처리'] = val_df['사람문장'].apply(preprocess_data)

In [ ]:
train_df

,Unnamed: 0,상황키워드,감정_대분류,감정_소분류,사람문장,시스템문장,사람문장_전처리
0,1,1,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,일은 왜 해도 해도 끝이 없을까? 화가 난다
1,1,1,분노,노여워하는,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,그냥 내가 해결하는 게 나아 남들한테 부담 주고 싶지도 않고
2,2,1,분노,노여워하는,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,최대한 지출을 억제해야겠어 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아
3,2,1,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나
4,3,1,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...
...,...,...,...,...,...,...,...
145949,51629,0,불안,걱정스러운,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해
145950,51629,0,불안,걱정스러운,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해
145951,51630,0,상처,배신당한,남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.,대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.,남편과 결혼한 지 사십 년이야 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져
145952,51630,0,상처,배신당한,사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야.,어떻게 하면 지금의 상황에 변화를 만들어낼 수 있을까요?,사람들을 만나는 것이 어려워 자꾸 사람들을 의심하게만 되고 말이야


In [ ]:
import random

dataset_train = []
for q, label in zip(train_df['사람문장_전처리'], train_df['상황키워드'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_train.append(data)

random.shuffle(dataset_train)

In [ ]:
dataset_test = []
for q, label in zip(val_df['사람문장_전처리'], val_df['상황키워드'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dataset_test.append(data)

random.shuffle(dataset_test)

In [ ]:
print(len(dataset_train))
print(dataset_train[0])
print(dataset_train[20000])
print(dataset_train[40000])
print(dataset_train[-1])

145954
['여자 친구가 헤어지자는 문자 이후로 내 얼굴도 안 보려고 해 벌써 혼자가 된 기분이야', '5']
['내가 치매래 나중에 남편까지 잊어버릴까 봐 두려워', '0']
['일단 입사했으니 친구 입장을 생각해서 삼 개월만 견디고 이직해야겠어', '0']
['과거에 내가 힘들때 많은 의지가됐거든 그러니까 이제부터라도 어떤 위기가 닥쳐도 염려없어', '4']


## 모델 학습

In [ ]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    """
    BERT 스타일의 데이터 변환 클래스.

    Parameters
    ----------
    tokenizer : BERTTokenizer
        문장을 토큰화하는 토크나이저.
    max_seq_length : int
        문장의 최대 시퀀스 길이.
    vocab : BERTVocab
        토큰 변환을 위한 어휘.
    pad : bool, default True
        문장을 최대 길이로 패딩할지 여부.
    pair : bool, default True
        문장 또는 문장 쌍을 변환할지 여부.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """
        문장 쌍 또는 단일 문장에 대한 변환 수행.

        Parameters
        ----------
        line: tuple of str
            입력 문자열. 문장 쌍의 경우 입력은 2개의 문자열로 이루어진 튜플이며:
            (text_a, text_b). 단일 문장의 경우 입력은 단일 문자열로 이루어진 튜플:
            (text_a,).

        Returns
        -------
        np.array: 'int32' 형식의 입력 토큰 ID, 형태 (batch_size, seq_length)
        np.array: 'int32' 형식의 유효 길이, 형태 (batch_size,)
        np.array: 'int32' 형식의 입력 토큰 타입 ID, 형태 (batch_size, seq_length)
        """

        # 유니코드로 변환
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # 'tokens_a' 및 'tokens_b'를 수정하여 총 길이가 지정된 길이보다 작아지도록 함
            # [CLS], [SEP], [SEP]에 해당하는 "- 3" 고려
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # [CLS] 및 [SEP]에 해당하는 "- 2" 고려
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # 문장의 길이
        valid_length = len(input_ids)

        if self._pad:
            # 시퀀스 길이까지 제로 패딩
            padding_length = self._max_seq_length - valid_length
            # 나머지는 패딩 토큰으로 채움
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 하이퍼 파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 12,   # 상황 클래스 수로 조정 클래스 12개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
# 모델 학습 전에 체크포인트 디렉토리를 지정합니다.
checkpoint_dir = '/content/drive/MyDrive/WCRC'
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_cosine_schedule_with_warmup
from tqdm import tqdm

# Early stopping 설정
best_test_acc = 0.0
no_improvement_count = 0
patience = 5

for e in range(num_epochs):
    # 훈련 과정
    model.train()
    train_acc = 0.0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e + 1, batch_id + 1, loss.data.cpu().numpy(),
                                                                       train_acc / (batch_id + 1)))

    avg_train_acc = train_acc / (batch_id + 1)
    print("epoch {} avg train acc {}".format(e + 1, avg_train_acc))

    # 검증 과정
    model.eval()
    test_acc = 0.0
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)

    avg_test_acc = test_acc / (batch_id + 1)
    print("epoch {} avg test acc {}".format(e + 1, avg_test_acc))

    # 모델 저장 및 early stopping
    if avg_test_acc > best_test_acc:
        best_test_acc = avg_test_acc
        final_model_filename = os.path.join(checkpoint_dir, 'situation_model.pth')
        torch.save(model.state_dict(), final_model_filename)
        print(f"Best model saved at {final_model_filename}")
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print("Early stopping after {} epochs of no improvement.".format(no_improvement_count))
        break

  0%|          | 1/2281 [00:00<37:52,  1.00it/s]

epoch 1 batch id 1 loss 1.1387509107589722 train acc 0.625


  9%|▉         | 201/2281 [02:02<21:10,  1.64it/s]

epoch 1 batch id 201 loss 1.3142964839935303 train acc 0.5862873134328358


 18%|█▊        | 401/2281 [04:03<19:02,  1.65it/s]

...

epoch 26 batch id 2201 loss 0.028387343510985374 train acc 0.9802362562471604


100%|██████████| 2281/2281 [22:59<00:00,  1.65it/s]


epoch 26 avg train acc 0.9801964598860149


100%|██████████| 281/281 [00:57<00:00,  4.90it/s]

epoch 26 avg test acc 0.6686499110320284
Early stopping after 5 epochs of no improvement.
